In [ ]:
# Exercise: Train a deep MLP on the MNIST dataset (you can load it using keras.datasets.mnist.load_data(). 
# See if you can get over 98% precision. Try searching for the optimal learning rate by using the approach 
# presented in this chapter (i.e., by growing the learning rate exponentially, plotting the loss, and 
# finding the point where the loss shoots up). Try adding all the bells and whistles—save checkpoints, 
# use early stopping, and plot learning curves using TensorBoard.